In [ ]:
import pandas as pd
import json
from google.colab import files

# Step 1: Upload JSON files
json_files = files.upload()

# Step 2: Initialize an empty list to collect data
data_list = []

# Step 3: Read each JSON file and append its content
for filename in json_files:
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
        if isinstance(data, list):
            data_list.extend(data)
        else:
            data_list.append(data)


Saving C325807_catalogue_structure.json to C325807_catalogue_structure.json


In [ ]:
# Step 4: Convert to DataFrame
rows = []

for d in data_list:
  rows += d.values()

data = pd.DataFrame(rows)
DescriptionFields = pd.json_normalize(data['DescriptionFields'])
scopeContent = pd.json_normalize(data['scopeContent'])
data = pd.concat([data, DescriptionFields, scopeContent], axis=1).drop(['DescriptionFields', 'scopeContent'], axis=1)

In [ ]:
# Load the spreadsheet into a pandas DataFrame
df = data

# Function to split a column by the first comma and create two new columns
def split_column_by_first_comma(df, column_name):
    if column_name in df.columns:
        split_data = df[column_name].astype(str).str.split(',', n=1, expand=True)
        df[f'{column_name}_name'] = split_data[0]
        # Check if the second part exists before trying to access it
        if split_data.shape[1] > 1:
            df[f'{column_name}_address'] = split_data[1]
            # Remove trailing full stop if it exists
            df[f'{column_name}_address'] = df[f'{column_name}_address'].astype(str).str.rstrip('.')
        else:
            df[f'{column_name}_address'] = None # Or any other placeholder for missing data
    return df

# Split 'copyright owner' column
df = split_column_by_first_comma(df, 'CopyrightOwner')

# Split 'copyrightauthor' column
df = split_column_by_first_comma(df, 'CopyrightAuthor')

# Reorder columns
original_columns = df.columns.tolist()
reordered_columns = []
for col in original_columns:
    reordered_columns.append(col)
    if col == 'CopyrightOwner':
        reordered_columns.append('CopyrightOwner_name')
        reordered_columns.append('CopyrightOwner_address')
    elif col == 'CopyrightAuthor':
        reordered_columns.append('CopyrightAuthor_name')
        reordered_columns.append('CopyrightAuthor_address')

# Remove duplicates caused by adding the new columns already
reordered_columns = [col for col in reordered_columns if col in original_columns or col.endswith('_name') or col.endswith('_address')]
reordered_columns = list(dict.fromkeys(reordered_columns)) # Remove duplicates while preserving order

# Filter out the original name and address columns from their original positions
reordered_columns = [col for col in reordered_columns if col not in ['CopyrightOwner_name', 'CopyrightOwner_address', 'CopyrightAuthor_name', 'CopyrightAuthor_address']]

# Insert the name and address columns after the original columns
final_columns = []
for col in df.columns:
    final_columns.append(col)
    if col == 'CopyrightOwner':
        final_columns.append('CopyrightOwner_name')
        final_columns.append('CopyrightOwner_address')
    elif col == 'CopyrightAuthor':
        final_columns.append('CopyrightAuthor_name')
        final_columns.append('CopyrightAuthor_address')

# Remove duplicates from the final list
final_columns = list(dict.fromkeys(final_columns))

# Select columns in the desired order
df = df[final_columns]

# Remove leading and trailing spaces from string columns
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.strip()

# Display the first few rows with the new columns and order
display(df.head())

,catalogueId,citableReference,coveringFromDate,coveringToDate,id,isParent,catalogueLevel,parentId,DescriptionText,image_list,...,CopyrightAuthor_name,CopyrightAuthor_address,FormCompleted,RegistrationStamp,PartiesToAgreement,DateOfAgreement,placeNames,description,ephemera,schema
0,-9357350,COPY 1/60/1,18830201,18830201,C15163751,False,7,C325807,1) 'Photograph of Lord Wolseley seated sideway...,[{'img_name': 'PDFs_COPY1_COPY-1-60_1_img0.jpg...,...,R and H Timperley,"Darwen, Lancashire",1 February 1883,1883 February 2,nan,nan,[],<scopecontent><p>1) 'Photograph of Lord Wolsel...,None,None
1,-9357351,COPY 1/60/2,18830204,18830204,C15163752,False,7,C325807,'Photograph from a drawing in colours of late ...,[{'img_name': 'PDFs_COPY1_COPY-1-60_1_img1.jpg...,...,Fred Bauer,"21 Scarborough Street, Goodman's Fields, London",4 February 1883[?],1883 February 3[?],nan,nan,[],<scopecontent>\r\n\t<p>'Photograph from a draw...,None,None
2,-9357352,COPY 1/60/3,18830204,18830204,C15163753,False,7,C325807,'Photograph from a drawing in colours of Miss ...,[{'img_name': 'PDFs_COPY1_COPY-1-60_1_img2.jpg...,...,Fred Bauer,"21 Scarborough Street, Goodman's Fields, London",4 February 1883[?],1883 February 3[?],nan,nan,[],<scopecontent>\r\n\t<p>'Photograph from a draw...,None,None
3,-9357353,COPY 1/60/4,18830204,18830204,C15163754,False,7,C325807,'Photograph from a drawing in colours of Princ...,[{'img_name': 'PDFs_COPY1_COPY-1-60_1_img3.jpg...,...,Fred Bauer,"21 Scarborough Street, Goodman's Fields, London",4 February 1883[?],1883 February 3[?],nan,nan,[],<scopecontent>\r\n\t<p>'Photograph from a draw...,None,None
4,-9357354,COPY 1/60/5,18830204,18830204,C15163755,False,7,C325807,'Photograph from a drawing in colours of Princ...,[{'img_name': 'PDFs_COPY1_COPY-1-60_1_img4.jpg...,...,Fred Bauer,"21 Scarborough Street, Goodman's Fields, London",4 February 1883[?],1883 February 3[?],nan,nan,[],<scopecontent>\r\n\t<p>'Photograph from a draw...,None,None


In [ ]:
# Save the DataFrame to an Excel file
output_excel_file = "/content/drive/My Drive/COPY 1_json_combined_split.xlsx"
df.to_excel(output_excel_file, index=False)

print(f"DataFrame saved to 'COPY 1_json_combined_split.xlsx'")

DataFrame saved to 'COPY 1_json_combined_split.xlsx'


In [ ]:
# Download the Excel file using google.colab.files
from google.colab import files

try:
  files.download(output_excel_file)
except Exception as e:
  print(f"Error downloading file: {e}")
  print("Please ensure you have executed the previous cell to create the file and that the file path is correct.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>